# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-06-17 18:29:42] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=31689, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=288703930, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1

[2025-06-17 18:29:57] Attention backend not set. Use fa3 backend by default.
[2025-06-17 18:29:57] Init torch distributed begin.


[2025-06-17 18:30:00] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 18:30:01] Load weight begin. avail mem=77.30 GB


[2025-06-17 18:30:02] Using model weights format ['*.safetensors']


...

[2025-06-17 18:30:03] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-06-17 18:30:03] INFO:     Started server process [787658]
[2025-06-17 18:30:03] INFO:     Waiting for application startup.
[2025-06-17 18:30:03] INFO:     Application startup complete.
[2025-06-17 18:30:03] INFO:     Uvicorn running on http://0.0.0.0:31689 (Press CTRL+C to quit)


[2025-06-17 18:30:04] INFO:     127.0.0.1:49812 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 18:30:04] INFO:     127.0.0.1:49820 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 18:30:04] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 18:30:06] INFO:     127.0.0.1:49824 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 18:30:06] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-17 18:30:09] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 18:30:09] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.54, #queue-req: 0


[2025-06-17 18:30:09] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 133.95, #queue-req: 0


[2025-06-17 18:30:10] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 133.75, #queue-req: 0
[2025-06-17 18:30:10] INFO:     127.0.0.1:46976 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-06-17 18:30:10] INFO:     127.0.0.1:46986 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-06-17 18:30:10] INFO:     127.0.0.1:46994 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-06-17 18:30:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 18:30:11] INFO:     127.0.0.1:47008 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-06-17 18:30:11] INFO:     127.0.0.1:47018 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-06-17 18:30:11] Cache flushed successfully!
[2025-06-17 18:30:11] INFO:     127.0.0.1:47034 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-06-17 18:30:11] Start update_weights. Load format=auto
[2025-06-17 18:30:11] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-06-17 18:30:11] Using model weights format ['*.safetensors']


[2025-06-17 18:30:11] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.54it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.53it/s]

[2025-06-17 18:30:11] Update weights end.
[2025-06-17 18:30:11] Cache flushed successfully!
[2025-06-17 18:30:11] INFO:     127.0.0.1:47050 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-06-17 18:30:11] Start update_weights. Load format=auto
[2025-06-17 18:30:11] Update engine weights online from disk begin. avail mem=41.50 GB


[2025-06-17 18:30:11] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-06-17 18:30:11] INFO:     127.0.0.1:47066 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-06-17 18:30:11] Child process unexpectedly failed with exitcode=9. pid=788370


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-17 18:30:18] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=32237, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1010056719, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, b

[2025-06-17 18:30:22] Downcasting torch.float32 to torch.float16.


[2025-06-17 18:30:29] Downcasting torch.float32 to torch.float16.


[2025-06-17 18:30:29] Overlap scheduler is disabled for embedding models.
[2025-06-17 18:30:29] Downcasting torch.float32 to torch.float16.
[2025-06-17 18:30:29] Attention backend not set. Use fa3 backend by default.
[2025-06-17 18:30:29] Init torch distributed begin.


[2025-06-17 18:30:30] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-17 18:30:34] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.64 GB


[2025-06-17 18:30:35] INFO:     Started server process [790615]
[2025-06-17 18:30:35] INFO:     Waiting for application startup.
[2025-06-17 18:30:35] INFO:     Application startup complete.
[2025-06-17 18:30:35] INFO:     Uvicorn running on http://0.0.0.0:32237 (Press CTRL+C to quit)


[2025-06-17 18:30:35] INFO:     127.0.0.1:48268 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 18:30:36] INFO:     127.0.0.1:48270 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 18:30:36] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 18:30:36] INFO:     127.0.0.1:48286 - "POST /encode HTTP/1.1" 200 OK
[2025-06-17 18:30:36] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-06-17 18:30:40] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-17 18:30:40] INFO:     127.0.0.1:57114 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-06-17 18:30:40] Child process unexpectedly failed with exitcode=9. pid=790956
[2025-06-17 18:30:40] Child process unexpectedly failed with exitcode=9. pid=790889


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-17 18:30:47] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=30576, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=321294700, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downlo

[2025-06-17 18:30:57] Overlap scheduler is disabled for embedding models.
[2025-06-17 18:30:58] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 18:30:58] Init torch distributed begin.


[2025-06-17 18:30:58] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 18:30:58] Load weight begin. avail mem=61.47 GB


[2025-06-17 18:30:58] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

[2025-06-17 18:31:02] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-06-17 18:31:02] Memory pool end. avail mem=44.64 GB


[2025-06-17 18:31:03] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=44.07 GB


[2025-06-17 18:31:03] INFO:     Started server process [793370]
[2025-06-17 18:31:03] INFO:     Waiting for application startup.
[2025-06-17 18:31:03] INFO:     Application startup complete.
[2025-06-17 18:31:03] INFO:     Uvicorn running on http://0.0.0.0:30576 (Press CTRL+C to quit)


[2025-06-17 18:31:03] INFO:     127.0.0.1:38326 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 18:31:04] INFO:     127.0.0.1:38328 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 18:31:04] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-06-17 18:32:08] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-17 18:32:08] INFO:     127.0.0.1:38340 - "POST /encode HTTP/1.1" 200 OK
[2025-06-17 18:32:08] The server is fired up and ready to roll!


[2025-06-17 18:32:22] INFO:     127.0.0.1:58712 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-06-17 18:32:22] Child process unexpectedly failed with exitcode=9. pid=793851
[2025-06-17 18:32:22] Child process unexpectedly failed with exitcode=9. pid=793773


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-17 18:32:29] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=39220, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=898716884, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, sleep_on_i

[2025-06-17 18:32:40] Attention backend not set. Use flashinfer backend by default.
[2025-06-17 18:32:40] Init torch distributed begin.


[2025-06-17 18:32:40] Init torch distributed ends. mem usage=0.00 GB


[2025-06-17 18:32:41] Load weight begin. avail mem=75.48 GB


[2025-06-17 18:32:41] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-06-17 18:32:48] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=44.28 GB


[2025-06-17 18:32:49] INFO:     Started server process [800492]
[2025-06-17 18:32:49] INFO:     Waiting for application startup.
[2025-06-17 18:32:49] INFO:     Application startup complete.
[2025-06-17 18:32:49] INFO:     Uvicorn running on http://0.0.0.0:39220 (Press CTRL+C to quit)


[2025-06-17 18:32:49] INFO:     127.0.0.1:46892 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-17 18:32:50] INFO:     127.0.0.1:46908 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-17 18:32:50] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-17 18:32:51] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1b-gpu-1/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-06-17 18:33:06] Resetting ExpertDistributionRecorder...
[2025-06-17 18:33:06] INFO:     127.0.0.1:46928 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-17 18:33:06] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-17 18:33:06] INFO:     127.0.0.1:46912 - "POST /generate HTTP/1.1" 200 OK
[2025-06-17 18:33:06] The server is fired up and ready to roll!


[2025-06-17 18:33:07] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 2.48, #queue-req: 0


[2025-06-17 18:33:08] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 35.50, #queue-req: 0


[2025-06-17 18:33:09] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: False, gen throughput (token/s): 36.22, #queue-req: 0


[2025-06-17 18:33:09] INFO:     127.0.0.1:33808 - "POST /generate HTTP/1.1" 200 OK


[2025-06-17 18:33:09] INFO:     127.0.0.1:41004 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-17 18:33:10] Write expert distribution to /tmp/expert_distribution_recorder_1750185190.7307286.pt
[2025-06-17 18:33:10] Resetting ExpertDistributionRecorder...
[2025-06-17 18:33:10] INFO:     127.0.0.1:41006 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-06-17 18:33:10] Child process unexpectedly failed with exitcode=9. pid=800795
[2025-06-17 18:33:10] Child process unexpectedly failed with exitcode=9. pid=800649
